# FE

* Calcular distancias correctamente

* Derivar velocidad, aceleración, días activos

* Etiquetar patrones activos/estacionarios

* Calcular distance_per_day, bearing, season, etc.

In [ ]:
import xarray as xr
from scipy.spatial import cKDTree

def load_seaice_dataset():
    """
    Carga y devuelve el dataset ambiental con concentración de hielo marino.
    """
    url = 'https://polarwatch.noaa.gov/erddap/griddap/nsidcG02202v4nh1day'
    ds = xr.open_dataset(url, engine='netcdf4')
    ds = ds.sel(time=slice('1985-01-01','2017-12-31'))
    return ds

def build_spatial_index(ds):
    """
    Construye un árbol espacial de coordenadas (lat, lon) para interpolación rápida.
    """
    lon_grid, lat_grid = np.meshgrid(ds['longitude'].values, ds['latitude'].values)
    coords = np.vstack([lat_grid.ravel(), lon_grid.ravel()]).T
    tree = cKDTree(coords)
    return tree, coords

def get_seaice_for_row(row, ds, tree, coords):
    """
    Devuelve el valor de cdr_seaice_conc para una fila dada.
    """
    date = pd.to_datetime(row["timestamp"]).date()
    try:
        sel_day = ds.sel(time=str(date))
        dist, idx = tree.query([row["adj_lat"], row["adj_lon"]], k=1)
        lat_idx, lon_idx = np.unravel_index(idx, sel_day['cdr_seaice_conc'][0].shape)
        return float(sel_day['cdr_seaice_conc'][0, lat_idx, lon_idx].values)
    except Exception:
        return np.nan


como usarla?

# 1. Cargar dataset y preparar árbol
ds = load_seaice_dataset()
tree, coords = build_spatial_index(ds)

# 2. Aplicar a tu dataframe
tqdm.pandas()
df["cdr_seaice_conc"] = df.progress_apply(lambda row: get_seaice_for_row(row, ds, tree, coords), axis=1)


 Notas importantes

    La latitud y longitud deben estar ajustadas (adj_lat, adj_lon)

    Esto puede tardar un poco en ejecutarse (~minutos), pero puedes guardar el resultado

    También puedes vectorizar más tarde usando interpolación xarray avanzada (si quieres rendimiento máximo)

In [ ]:
# Asegúrate de que el bearing esté ordenado por tiempo por osa
df = df.sort_values(["UniqueAnimalID", "timestamp"])

df["bearing_prev"] = df.groupby("UniqueAnimalID")["bearing_nc"].shift()
df["bearing_change"] = (df["bearing_nc"] - df["bearing_prev"]).abs()
df["bearing_change"] = df["bearing_change"] % 360  # Corregir saltos circulares
df["bearing_change"] = df["bearing_change"].apply(lambda x: 360 - x if x > 180 else x)


In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df["bearing_change"].dropna(), bins=36, color="#1f77b4")
plt.title("Distribución de cambios de dirección (bearing_change)")
plt.xlabel("Cambio en dirección (°)")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()
